In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings

#ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
selectedColumns = ['ACCIDENT DATE', 'ACCIDENT TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'NUMBER OF PERSONS KILLED']

In [ ]:
df = pd.read_csv("nypd-motor-vehicle-collisions.csv", low_memory=False)

In [ ]:
import numpy as np
df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
df = df[selectedColumns]
df.head()

In [ ]:
df = df[df['LATITUDE'].notna()]


In [ ]:
df = df[df['LONGITUDE'].notna()]

In [ ]:
df["BOROUGH"].value_counts()

In [ ]:
df['BOROUGH'].value_counts().plot(kind='bar', title='Location Borough');

In [ ]:
df["ZIP CODE"].value_counts()

In [ ]:
df["ZIP CODE"].value_counts()[:10].plot(kind='bar', title="Top 10 Zip Code")

In [ ]:
import folium
from folium.plugins import MarkerCluster 

In [ ]:
map = folium.Map(location=[40.693943, -73.985880], default_zoom_start=15)

In [ ]:
folium.Marker(
    location=[40.693943, -73.985880],
    popup='Welcome to <b>NEW YORK CITY</b>',
).add_to(map)

In [ ]:
df['KILLED_INJURED'] = df.groupby(df['ZIP CODE'])['NUMBER OF PERSONS KILLED'].transform('count')
df.head()

In [ ]:
map.choropleth(geo_data="nyc-zip-codes.geojson",
             data=df,
             columns=['ZIP CODE', 'KILLED_INJURED'],
             key_on='feature.properties.postalCode', 
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Person Injured and Killed')

In [ ]:
marker_cluster = MarkerCluster().add_to(map)
import math
try:
    for i in range(df.shape[0]):
        location = [df['LATITUDE'][i],df['LONGITUDE'][i]]
        tooltip = "Zipcode:{}<br> Borough: {}<br> Click for more".format(df["ZIP CODE"][i], df['BOROUGH'][i])
        folium.Marker(location,
                    popup="""
                        <i>ZIP CODE: </i> <br> <b>{}</b> <br> 
                        <i>BOROUGH: </i><b><br>{}</b><br>
                        <i>KILLED AND INJURED: </i><b><br>{}</b><br>""".format(
                    df['ZIP CODE'][i], 
                    df['BOROUGH'][i],
                    df['KILLED_INJURED'][i]), 
                tooltip=tooltip).add_to(marker_cluster)
except KeyError:
    pass

In [ ]:
map